# Advanced Operations

In [1]:
import gtn
import nb_utils
nb_utils.init()

## Intersect

The intersection of two acceptors is the set of strings which is accepted by both of them. The score of a path in the intersected graph is the sum of the score of the path in each of the input graphs. More formally the language of the intersected graph is given by $\{ x \mid x \in \mathcal{L}(A_1) \;\land\; x \in \mathcal{L}(A_2)\}$.

The analagous operation for transducers is the composition, which we will discuss in more detail in the next section. In most machine learning applications, intersect and compose tend to be the primary operations used to compute more complex graphs from simpler input graphs. Thus gaining a deeper understanding of these two operations is worth the time.

In [2]:
symbols = {0: 'a', 1: 'b', 2: 'c'}
isymbols = {0: 'a', 1: 'b', 2: 'c'}
osymbols = {0: 'x', 1: 'y', 2: 'z'}

g1 = gtn.Graph()
g1.add_node(start=True)
g1.add_node(accept=True)
g1.add_arc(src_node=0, dst_node=0, label=0)
g1.add_arc(src_node=0, dst_node=1, label=1)

g2 = gtn.Graph()
g2.add_node(start=True)
g2.add_node()
g2.add_node(accept=True)
g2.add_arc(src_node=0, dst_node=1, label=0)
g2.add_arc(src_node=0, dst_node=1, label=1)
g2.add_arc(src_node=0, dst_node=1, label=2)
g2.add_arc(src_node=1, dst_node=2, label=0)
g2.add_arc(src_node=1, dst_node=2, label=1)
g2.add_arc(src_node=1, dst_node=2, label=2)

gtn.draw(g1, "figures/nb/intersect_1.svg", isymbols=symbols)
gtn.draw(g2, "figures/nb/intersect_2.svg", isymbols=symbols)

We'll use the two graphs below to illustrate a general algorithm for computing the intersection. In this case, the intersected graph is easy to see by inspecting the two graphs. The only string which is recognized by both graphs is the string $ab$, hence the intersected graph is the graph which recognizes $ab$.

<div class="figure">
  <div class="img">
    <img src="figures/intersect_1.svg"/>
  </div>
  <div class="img" style="padding-top:23px;">
    <img src="figures/intersect_2.svg"/> 
  </div>
</div>

The general algortihm relies on a queue to explore pairs of states from the two input graphs. Pseudocode is given below.

```C++
function intersect(g1, g2) {
    queue = Queue()
    new_graph = Graph()
    
    // Add all pairs of start states to the queue. For each start state pair
    // add a corresponding start state to the intersected graph.
    for s1, s2 in start_state_pairs(g1, g2) {
        new_graph.add_state((s1, s2))
        queue.push((s1, s2))
    }
    
    while not queue.empty() {
        (u1, u2) = queue.pop()
        // Get all pairs of outgoing arcs from states u1 and u2 with
        // matching labels 
        for arc1, arc2 in label_matching_arcs(u1, u2) {
            v1 = arc1.dst_node()
            v2 = arc2.dst_node()
            
            // If we haven't visited the state pair (v1, v2) yet, then add it
            // to the queue and create a corresponding state in the intersected
            // graph. The new state in the intersected graph is an accept state
            // if both v1 and v2 are accept states.
            if not_visited((v1, v2)) {
                queue.push((v1, v2))
                new_graph.add_state((v1, v2))
            }
            
            // Add an arc to the new graph from (u1, u2) to (v1, v2) with the
            // label from the matching arcs and a weight given by the sum of
            // the weights of the matching arcs.
            label = arc1.label()
            weight = acr1.weight() + arc2.weight()
            new_graph.add_arc((u1, u2), (v1, v2), label, weight)
        }
    }
    return new_graph
}
```

Some of the steps of the intersect algorithm on the two input graphs above are shown in the sequence of figures below.

<table>
  <tr>
      <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_1_g1.svg"/></td>
      <td><img src="figures/nb/intersect_step_1_g2.svg"/></td>
      <td><img src="figures/nb/intersect_step_1_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The starts states in the input graphs are $0$ and $0$. So we add the start state $(0, 0)$ to the intersected graph and to the queue to be explored.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_2_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_2_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_2_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The first pair of outgoing arcs match on the label $a$. This means the downstream state $(0, 1)$ is reachable in the intersected graph. So we add $(0, 1)$ to the intersected graph and to the queue to be explored.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_3_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_3_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_3_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The next matching pair outgoing arcs match on the label $b$. Also we haven't visited the downstream state $(1, 1)$ yet. So we add $(1, 1)$ to the intersected graph and to the queue to be explored.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_4_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_4_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_4_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The arc in the second input graph with label $c$ does not have a matching arc from state $0$ in the first input graph, so it is ignored.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_5_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_5_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_5_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">At this point we have considered all the matching outgoing arcs from $(0, 0)$ so we now move on to the next state pair in the queue, $(0, 1)$. From $(0, 1)$, a pair of outgoing arcs match on the label $a$. The downstream state $(0, 2)$ is new so we add it to the intersected graph and to the queue.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_6_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_6_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_6_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The next pair of matching outgoing arcs match on the label $b$ and lead to the state pair $(1, 2)$. We add $(1, 2)$ to the queue and to the interescted graph. Also, since $1$ is an accept state in the first graph and $2$ is an accept state in the second graph, $(1, 2)$ is an accept state in the intersected graph.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_7_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_7_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_7_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">Again, the arc in the second input graph with label $c$ does not have a matching arc from state $0$ in the first input graph, so it is ignored.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_8_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_8_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_8_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The next state pair in the queue is $(1, 1)$. There are no arcs leaving state $1$ in either input graph, and $(1, 1)$ is not an accept state in the intersected graph, so it is a dead end. We can remove $(1, 1)$ and its incoming arcs from the intersected graph.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_9_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_9_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_9_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The next state pair in the queue is $(0, 2)$. Again, there are no matching arcs leaving this state pair, and $(0, 2)$ is not an accept state in the intersected graph, hence it is a dead end. We can also remove $(0, 2)$ and its incoming arcs from the intersected graph.</div></td>
  </tr>
  <tr>
    <td style="padding-bottom:27px;"><img src="figures/nb/intersect_step_10_g1.svg"/></td>
    <td><img src="figures/nb/intersect_step_10_g2.svg"/></td>
    <td><img src="figures/nb/intersect_step_10_g_out.svg"/></td>
  </tr>
  <tr>
      <td colspan="3"><div class="table_caption" markdown="span">The next state pair in the queue is $(1, 2)$. There are no matching arcs for this state pair, but it is an accept state in the intersected graph so we have to keep it. At this point the queue is empty so the algorithm terminates and we are left with the complete intersected graph.</div></td>
  </tr>
</table>

The pseudocode does not handle $\epsilon$ transitions. The challenge with including $\epsilon$ transitions in intersect and compose is discussed later.

Another potential issue with the pseudocode above is that the output graph it constructs is not *trim*. An automata is *trim* if every state in the graph is part of a path which starts in a start state and terminates in an accept state. In short, the graph does not have any useless states. In the standard terminology, a state is said to be *accessible* if it can be reached from a start state and *coaccessible* if an accept state can be reached from it. A graph is trim if every state is both accessible and coaccessible. 

Every state in the pseudocode above will be accessible, but not necessarily coaccessible, so the graph may not be trim. However, the graph will still be correct, so this is primarily an issue of representation size. With some additional work the constructed graph can be kept trim during the operation of the algorithm. Another alternative is to construct a trim graph from the non-trim graph as a post-processing step.

---

### Example

Compute the intersection of the two graphs below. Make sure to update the arc weights correctly in the intersected graph.

In [5]:
g1 = gtn.Graph()
g1.add_node(start=True)
g1.add_node(accept=True)
g1.add_arc(src_node=0, dst_node=0, label=0)
g1.add_arc(src_node=0, dst_node=1, label=1)
g1.add_arc(src_node=1, dst_node=1, label=2)
g1.set_weights([1, 2, 3])

g2 = gtn.Graph()
g2.add_node(start=True)
g2.add_node()
g2.add_node()
g2.add_node(accept=True)
g2.add_arc(src_node=0, dst_node=1, label=0)
g2.add_arc(src_node=0, dst_node=1, label=1)
g2.add_arc(src_node=0, dst_node=1, label=2)
g2.add_arc(src_node=1, dst_node=2, label=0)
g2.add_arc(src_node=1, dst_node=2, label=1)
g2.add_arc(src_node=1, dst_node=2, label=2)
g2.add_arc(src_node=2, dst_node=3, label=0)
g2.add_arc(src_node=2, dst_node=3, label=1)
g2.add_arc(src_node=2, dst_node=3, label=2)
g2.set_weights([1, 2, 3, 2, 3, 4, 3, 4, 5])

gtn.draw(g1, "figures/nb/intersect_example_1.svg", isymbols=symbols)
gtn.draw(g2, "figures/nb/intersect_example_2.svg", isymbols=symbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/intersect_example_1.svg"/>
  </div>
  <div class="img" style="padding-top:28px;">
    <img src="figures/nb/intersect_example_2.svg"/> 
  </div>
  <div class="caption" markdown="span">
  The automata on the left recognizes $a^*bc^*$, the automata on the right recognizes all three letter combinations of the alphabet $\{a, b, c\}$.
  </div>
</div>

In [12]:
g_out = gtn.intersect(g1, g2)
gtn.draw(g_out, "figures/nb/intersect_example.svg", isymbols=symbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/intersect_example.svg"/>
  </div>
  <div class="caption" markdown="span">
     The intersected graph accepts the strings $aab$, $abc$, and $bcc$.
  </div>
</div>

---

## Compose

The composition is a straightforward generalization of the intersection from acceptors to transducers. Assume the first inpuut graph transduces the string $x$ to the string $y$ and the second graph transduces $y$ to $z$. The composed graph must then transducer $x$ to $z$. Intuitively, composition let's us construct mappings between different representations. For example, suppose we know how phonemes map to words. 

From an implementation standpoint the compose and intersect algorithms are almonst identical. The two minor differences are 1) the way that labels are matched in the input graph and 2) the labels of the new arcs in the composed graph. Arcs in a transducer have both input and output labels. We match the output arc label from the first input graph to the input arc label from the second input graph. This means that compose, unlike intersect, is not commutative since the order of the two graphs makes a difference.

The input label of a new arc in the composed graph is the input label of the corresponding arc in the first input graph. The output label of a new arc in the composed graph is the output label of the corresponding arc in the second input graph. Assume we have two arcs, the first with label $x_1\!:\!x_2$, and the second with label $y_1\!:\!y_2$. If $x_2 = y_1$, then the arcs are considered a match. The new arc will have the label $x_1\!:\!y_2$.

Consider matrix mutiplication as an analogy. When multiplying two matrices the inner dimensions must match and the dimensions of the output matrix are the outer dimensions of the two input matrices. In the same way the inner labels of the two arcs must match and the resulting output label are the outer labels of the two arcs.

---

### Example

Compute the composition of the following two graphs.

In [6]:
g1 = gtn.Graph()
g1.add_node(start=True)
g1.add_node(accept=True)
g1.add_arc(src_node=0, dst_node=0, ilabel=0, olabel=0, weight=1)
g1.add_arc(src_node=0, dst_node=1, ilabel=1, olabel=1, weight=2)
g1.add_arc(src_node=1, dst_node=1, ilabel=2, olabel=2, weight=3)

g2 = gtn.Graph()
g2.add_node(start=True)
g2.add_node()
g2.add_node()
g2.add_node(accept=True)
g2.add_arc(src_node=0, dst_node=1, ilabel=0, olabel=0)
g2.add_arc(src_node=0, dst_node=1, ilabel=0, olabel=1)
g2.add_arc(src_node=0, dst_node=1, ilabel=1, olabel=2)
g2.add_arc(src_node=1, dst_node=2, ilabel=0, olabel=0)
g2.add_arc(src_node=1, dst_node=2, ilabel=1, olabel=1)
g2.add_arc(src_node=1, dst_node=2, ilabel=2, olabel=2)
g2.add_arc(src_node=2, dst_node=3, ilabel=1, olabel=0)
g2.add_arc(src_node=2, dst_node=3, ilabel=2, olabel=1)
g2.add_arc(src_node=2, dst_node=3, ilabel=2, olabel=2)
g2.set_weights([1, 2, 3, 3, 2, 1, 2, 1, 3])

gtn.draw(g1, "figures/nb/compose_example_1.svg", isymbols=isymbols, osymbols=osymbols)
gtn.draw(g2, "figures/nb/compose_example_2.svg", isymbols=osymbols, osymbols=isymbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/compose_example_1.svg"/>
  </div>
  <div class="img" style="padding-top:28px;">
    <img src="figures/nb/compose_example_2.svg"/> 
  </div>
  <div class="caption" markdown="span">
    Compute the composition of the two transducers.
  </div>
</div>

In [7]:
g_out = gtn.compose(g1, g2)
gtn.draw(g_out, "figures/nb/compose_example.svg", isymbols=isymbols, osymbols=isymbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/compose_example.svg"/>
  </div>
  <div class="caption" markdown="span">
    The composed graph.
  </div>
</div>

---

### Intersection and Composition with $\epsilon$

The basic implementation of intersection and composition we have discussed so far doesn't extend to $\epsilon$ transitions. Allowing $\epsilon$ transitions in these algorithms makes them more complicated. In this section we'll illustrate the challenges with the naive approach but we will only discuss at a high-level how to actually include $\epsilon$ transitions in the algorithms.

First, consider the simpler case when only the first input graphs has $\epsilon$ transitions. In this case, whenever we encounter an outgoing $\epsilon$ transition from a state in the first graph we can optionally traverse it without matching a corresponding arc in the second graph. Consider the two subgraphs below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/epsilon_intersect_1.svg"/>
  </div>
  <div class="img" style="padding-top:28px;">
    <img src="figures/nb/epsilon_intersect_2.svg"/> 
  </div>
  <div class="caption" markdown="span">
    Compute the composition of the two transducers.
  </div>
</div>

Suppose we are currently looking for outgoing arcs with matching labels from the state $(0, 0)$. When we find a matching pair, we add the new state in the intersected graph and add a corresponding arc. In the $\epsilon$-free case, we only explore the arc's with label $a$. The state $(1, 1)$ is added to the intersected graph and the queue to be explored. The state $(0, 0)$ is connected to the state $(1, 1)$ with an arc with label $a$ and weight $2$. 

Since state $0$ in the first graph has an outgoing $\epsilon$, we can optionally traverse it without traversing any arc in the second graph. In this case, we add the state $(2, 0)$ to the intersected graph and to the queue to be explored. We also add an arc from the state $(0, 0)$ to the state $(2, 0)$ in the intersected graph with a label $\epsilon$ and a weight of $3$.

---

### Example

Compute the intersection of the two graphs below.

In [20]:
g1 = gtn.Graph()
g1.add_node(start=True)
g1.add_node()
g1.add_node()
g1.add_node(accept=True)
g1.add_arc(src_node=0, dst_node=1, label=0)
g1.add_arc(src_node=1, dst_node=2, label=gtn.epsilon)
g1.add_arc(src_node=2, dst_node=3, label=1)
g1.set_weights([0.5, 1.5, 2.5])

g2 = gtn.Graph()
g2.add_node(start=True)
g2.add_node(accept=True)
g2.add_arc(src_node=0, dst_node=0, label=0)
g2.add_arc(src_node=0, dst_node=1, label=1)
g2.set_weights([0.5, 1.5])

gtn.draw(g1, "figures/nb/epsilon_intersect_example_1.svg", isymbols=symbols)
gtn.draw(g2, "figures/nb/epsilon_intersect_example_2.svg", isymbols=symbols)

<div class="figure">
  <div class="img" style="padding-top:37px;">
    <img src="figures/nb/epsilon_intersect_example_1.svg"/>
  </div>
  <div class="img">
    <img src="figures/nb/epsilon_intersect_example_2.svg"/> 
  </div>
  <div class="caption" markdown="span">
    Compute the intersection of the two acceptors.
  </div>
</div>

In [14]:
g = gtn.intersect(g1, g2)
gtn.draw(g, "figures/nb/epsilon_intersect_example.svg", isymbols=symbols)

The intersected graph is below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/epsilon_intersect_example.svg"/>
  </div>
  <div class="caption" markdown="span">
    Compute the intersection of the two acceptors.
  </div>
</div>

---

The trickier case to handle is when both graphs have $\epsilon$ transitions. If we optionally explore outgoing $\epsilon$ arcs in each graph, then we will end up with too many paths in the intersection. Suppose we are given the two graphs below, each of which has an $\epsilon$ transition.

<div class="figure">
  <div class="img" style="padding-top:42px;">
    <img src="figures/nb/epsilon_intersect_both_1.svg"/>
  </div>
  <div class="img">
    <img src="figures/nb/epsilon_intersect_both_2.svg"/>
  </div>
  <div class="caption" markdown="span">
    Compute the intersection of the two acceptors.
  </div>
</div>

If we compute the intersection of the above two graphs, optionally following $\epsilon$ transitions as we encounter them, then we end up with the graph below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/epsilon_intersect_both.svg"/>
  </div>
  <div class="caption" markdown="span">
    Compute the intersection of the two acceptors.
  </div>
</div>

The language of this graph is correct. It accepts the string $ab$ which is the only string in the intersection. However, the weight it assigns to the string $ab$ is incorrect. Each individual path has the correct weight, but there are three paths for $ab$. The final weight will receive three contributions, one from each path, instead of a single contribution from one path. The solution to this problem is to choose only one of the three paths and avoid the inadvertant redundancy. For example we could keep the bottom path and ignore the top two as in the graph below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/epsilon_intersect_both_correct.svg"/>
  </div>
  <div class="caption" markdown="span">
    Compute the intersection of the two acceptors.
  </div>
</div>

## Forward and Viterbi
$\DeclareMathOperator*{\LSE}{\textrm{LSE}}$
The forward score and the viterbi score take a graph as input and return a single scalar result. The *forward score* is the accumulation of the weights of all possible paths from a start state to an accept state in the graph. The weight of the highest scoring path is the *Viterbi score*, and the path itself is the *Viterbi path*.

---

### Shortest Distance (aside)

In some descriptions of weighted automata, the forward and Viterbi score are introduced as shortest distance algorithms under a respective semiring. The forward score corresponds to the log semiring and the Viterbi score corresponds to the tropical semiring. This is a more general perspective, and useful if you intend to use other semirings. However, we only need the log and tropical semirings in all of the applications we study, so we will restrict the description to the more specific forward and Viterbi score.

---

Let's start with a couple of examples to show exactly what we are trying to compute, then we will go through a more general algorithm for forward and Viterbi scoring. For the forward and Viterbi score, we will restrict the graphs to be acyclic, meaning no self-loops of cycles. Under certain technical conditions a graph with cycles can admit a computable forward and Viterbi score, but we won't go into detail there.

In [10]:
fsa = gtn.Graph()
fsa.add_node(start=True)
fsa.add_node(start=True)
fsa.add_node()
fsa.add_node(accept=True)
fsa.add_arc(src_node=0, dst_node=1, label=0)
fsa.add_arc(src_node=0, dst_node=2, label=1)
fsa.add_arc(src_node=1, dst_node=2, label=2)
fsa.add_arc(src_node=2, dst_node=3, label=0)
fsa.set_weights([1.1, 3.2, 1.4, 2.1])
gtn.draw(fsa, "figures/nb/score_example.svg", isymbols=symbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/score_example.svg"/>
  </div>
  <div class="caption" markdown="span">
    TODO
  </div>
</div>

The graph above has three possible paths from the start states to the accept state. The paths and their scores are:

- State sequence $0 \rightarrow 1 \rightarrow 2 \rightarrow 3$ with score 4.6
- State sequence $0 \rightarrow 2 \rightarrow 3$ with score 5.3
- State sequence $1 \rightarrow 2 \rightarrow 3$ with score 3.5

The Viterbi score is the maximum over the individual path scores, in this case $\max\{4.6, 5.3, 3.5\} = 5.3$. The Viterbi path is the sequence of labels which correspond to the arcs contributing to the Viterbi score. We represent the Viterbi path as a simple linear graph as below. The forward score is the *log-sum-exp* over all the path scores, in this case $\LSE(4.6, 5.3, 3.5) = 5.81$.

In [3]:
viterbi_path = gtn.viterbi_path(fsa)
gtn.draw(viterbi_path, "figures/nb/viterbi_path.svg", isymbols=symbols)

NameError: name 'fsa' is not defined

<div class="figure">
  <div class="img">
    <img src="figures/nb/viterbi_path.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

We computed the forward and Viterbi score by listing all possible paths, compute their individual weights and then accumulating either with the $\max$ or the $LSE$ operations. This approach won't scale to larger graphs because the number of paths in such graphs can grow combinatorially. Instead, we will construct a much more efficient dynamic programming algorithm which works for both the forward and Viterbi score.

To construct the dynamic programming algorithm, we use the following recurions. Consider a state $v$ and let $e_i$ for $i=1, \ldots, k$ be the set of arcs for which $v$ is the destination node. For a given arc $e$ we let $\textrm{source}(e)$ denote the source node for that arc. The score of all paths which start at a start state and terminate at node $v$ can be constructed from the score of all paths which start at a start state and terminate at the states $\textrm{source}(e_i)$ and the arc weights $w(e_i)$. For the Viterbi score, the recursion is:
$$
s_v = \max_{i=1}^k \left( s_{\textrm{source}(e_i)} + w(e_i) \right)
$$
where $s_v$ is the score of all paths starting at a start state terminating at state $v$, and $s_{\textrm{source}(e_i)}$ is the score of all paths starting at a start state terminating at state $\textrm{source}(e_i)$. The recursion is shown graphically below. In the figure, we can compute the viterbi score of the state $3$ by taking the maximum over the weight of all incoming arcs plus their source node scores.

<div class="figure">
  <div class="img">
    <img src="figures/nb/viterbi_recursion.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

To arrive at the overall Viterbi score for the graph we take the max over the Viterbi scores of the accept states, $\max_a s_a$ where $a$ is an accept state.

To compute the forward score we use the same idea with an $\LSE$ in place of the $\max$.
$$
s_v = \LSE_{i=1}^k \left( s_{\textrm{source}(e_i)} + w(e_1) \right),
$$
and the final score is $\LSE_a s_a$.

For both the forward and Viterbi score, the recursion works because the $\LSE$ and $\max$ operations admit a simple decomposition of the score for all paths terminating at a given state. Suppose, as in the graph below, we have a state $v$ for which we want to compute the score $s_v$. Suppose also that $v$ has only one incoming arc from state $u$ and that state $u$ has three paths terminating with the given scores $p_1$, $p_2$, and $p_3$. We can extend each of the three paths terminating at $u$ to $v$ by adding the arc between them, so there are three paths terminating at $v$ as well.

<div class="figure">
  <div class="img">
    <img src="figures/dp_recursion.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

First suppose we want to compute the Viterbi score at $v$. The Viterbi score is the maximum of the weights of all three paths terminating at $v$, namely $s_v = \max \{w + p_1, w + p_2, w + p_3\}$. However, assume we are given the Viterbi score, $s_u$, of paths terminating at $u$. In this case we can compute $s_v = w + s_u$. This decomposition is valid, which we can prove by noting that two scores are equal:

$$
w + s_u = w + \max \{p_1, p_2, p_3 \} = \max \{w + p_1, w + p_2, w + p_3\}.
$$

The same decomposition works for the forward score and the $\LSE$ operation:

$$
\begin{align}
s_v &= w + s_u  \\
&= w + \log \left(e^{p_1} + e^{p_2} + e^{p_3}\right) \\
&= \log e^w + \log \left(e^{p_1} + e^{p_2} + e^{p_3}\right) \\
&= \log e^w \left(e^{p_1} + e^{p_2} + e^{p_3}\right) \\
&= \log \left(e^{w + p_1} + e^{w + p_2} + e^{w + p_3}\right) \\
&= s_v
\end{align}
$$

Here we assume only three paths terminating at $u$ for simplicity. In each case, the argument is easily extended to an arbitrary number of paths.


### Viterbi Path

A Viterbi path is a path for which the Viterbi score is attained. We can compute one of the Viterbi paths (usually there will only be one) with a straightforward extension of the Viterbi shortest distance algorithm. At each state when we compute the score, we also maintain a backpointer to the arc which resulted in the maximum score. When the algorithm terminates, we can trace the backpointers to the start state and extract the Viterbi path.

An example of this can be seen in the graph below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/viterbi_path_back.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

Each state in the graph is labelled with the Viterbi score over all paths terminating at that state. The red arcs denote are the arcs which result in the maximum score for the state that they point to. In order to compute the Viterbi path, we simply trace the red arcs back from the accept state. In this case the Viterbi path state sequence is $0 \rightarrow 2 \rightarrow 4 \rightarrow 6$ with arc labels $b$, $c$, and $b$.

---

### Example

Compute the Viterbi path of the graph below.

In [4]:
fsa = gtn.Graph()
fsa.add_node(start=True)
fsa.add_node()
fsa.add_node()
fsa.add_node(accept=True)

for i in range(3):
    for j in range(3):
        fsa.add_arc(i, i+1, j)
fsa.set_weights([0.3, 0.5, 0.2, 0.4, 0.1, 0.7, 0.4, 0.9, 0.1])
gtn.draw(fsa, "figures/nb/viterbi_path_example_input.svg", isymbols=symbols)

<div class="figure">
  <div class="img">
    <img src="figures/nb/viterbi_path_example_input.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

In [5]:
gtn.draw(gtn.viterbi_path(fsa), "figures/nb/viterbi_path_example.svg", isymbols=symbols)

The Viterbi path is shown in the graph below.

<div class="figure">
  <div class="img">
    <img src="figures/nb/viterbi_path_example.svg"/>
  </div>
  <div class="caption" markdown="span">
      TODO
  </div>
</div>

---